In [4]:
from utils_logitlens import get_parser, load_all_generations_logitlens, LogitLens
import matplotlib.pyplot as plt
import os
import numpy as np

In [46]:
# save_dir = "./generated_hidden_states_logitlens"
# hs_filename = "logitlens_hidden_states__model_name_gpt-j__parallelize_False__dataset_name_amazon_polarity__split_test__prompt_idx_0__batch_size_4__num_examples_100__use_decoder_False__layer_-1__all_layers_True__token_idx_-1.npy"
# y_filename = "logitlens_labels__model_name_gpt-j__parallelize_False__dataset_name_amazon_polarity__split_test__prompt_idx_0__batch_size_4__num_examples_100__use_decoder_False__layer_-1__all_layers_True__token_idx_-1.npy"
hs_all = np.load("generated_hidden_states_logitlens_clean/num_context_0.npy")
y = np.load("generated_hidden_states_logitlens_clean/num_context_0_labels.npy")
# hs_all, y = load_all_generations_logitlens(generation_args)

In [6]:
from transformers import GPTJForCausalLM, AutoTokenizer
import torch

device = "cuda"
model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [8]:
print(hs_all.shape)
print(y.shape)

(500, 4096, 29)
(500,)


In [9]:
# get unembedding matrix of the model
embedding_matrix = model.get_input_embeddings().weight
# I think the embedding and un-embedding matrices of GPT-J are the same?
# yes they are: https://ai.stackexchange.com/questions/36192/how-does-gpt-use-the-same-embedding-matrix-for-both-input-and-output#:~:text=Yes%2C%20GPT%20uses%20the%20same,See%20here.

In [28]:
layernorm = model.state_dict()['transformer.ln_f.weight']
print(layernorm.shape)

torch.Size([4096])


In [35]:
norm_bias = model.state_dict()['transformer.ln_f.bias']
print(norm_bias.shape)

torch.Size([4096])


In [24]:
embedding_matrix.shape

torch.Size([50400, 4096])

In [25]:
tokenizer.vocab_size
# why is the embedding dimension larger than the vocab size?

50257

In [26]:
# try to perform logit lens on the first layer
first_hs = hs_all[:, :, 0]
print(first_hs.shape)

(500, 4096)


In [36]:
norm_l0 = torch.einsum('ij, j -> ij', torch.Tensor(first_hs).to(device), layernorm)
norm_l0 += norm_bias
logits_l0 = torch.einsum('ij, kj -> ki', embedding_matrix.float(), norm_l0)
logits_l0.shape

torch.Size([500, 50400])

In [37]:
# get the id of the max logit for each sample
ids_l0 = torch.argmax(logits_l0, dim=1)
ids_l0.shape

torch.Size([500])

In [38]:
# get the actual tokens
tokenizer.batch_decode(ids_l0)

[' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 ' the',
 

In [39]:
tokenizer.encode("positive")[0]

24561

In [40]:
# actually we want the difference between the correct anser and the incorrect answer?
def logits_to_logit_diff(logits, pos_ans="positive", neg_ans="negative"):
    # model.to_single_token maps a string value of a single token to the token index for that token
    # If the string is not a single token, it raises an error.
    # correct_index = model.to_single_token(correct_answer)
    # incorrect_index = model.to_single_token(incorrect_answer)

    pos_index = tokenizer.encode(pos_ans)
    neg_index = tokenizer.encode(neg_ans)

    return logits[:, pos_index] - logits[:, neg_index]


In [41]:
logits_to_logit_diff(logits_l0)

tensor([[0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0.0676],
        [0

In [42]:
def get_prediction(diffs):
    return torch.where(diffs > 0, 1, 0)

In [45]:
predictions = get_prediction(logits_to_logit_diff(logits_l0))
predictions

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],

In [44]:
y

array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,

In [46]:
predictions.squeeze()

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [47]:
def binary_accuracy(tensor1, tensor2):
    assert tensor1.shape == tensor2.shape, "Input tensors must have the same shape"
    correct_predictions = torch.eq(tensor1, tensor2).sum().item()
    total_elements = tensor1.numel()
    accuracy = correct_predictions / total_elements
    return accuracy
binary_accuracy(predictions.squeeze(), torch.Tensor(y).to(device))

0.468

: 

# Cleaning the evaluation dataframes

In [114]:
import pandas as pd
logit_diffs = pd.read_csv("figures/posvals_10context_norm")
logit_diffs

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,...,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159,3.411159
1,1,-1.441616,-1.566062,-1.602333,-1.448177,-1.618578,-1.480112,-1.533273,-1.517702,-1.568027,...,-1.563239,-1.556352,-1.457308,-1.480310,-1.467292,-1.539434,-1.558260,-1.658414,-1.521465,-1.540361
2,2,-3.118092,-3.236216,-3.286282,-3.232544,-3.253272,-3.232137,-3.169936,-3.244460,-3.273130,...,-3.189449,-3.277977,-3.230845,-3.239767,-3.176566,-3.256016,-3.269503,-3.175600,-3.215193,-3.231422
3,3,26.490910,26.448448,26.365700,26.523554,26.057621,25.930210,26.292116,26.376410,26.108385,...,26.236618,26.258892,26.422932,26.607695,26.478043,26.300419,26.154408,26.054653,26.567352,26.354519
4,4,26.443226,26.418404,26.263874,26.482958,25.997768,25.854660,26.219578,26.219560,26.098999,...,26.213343,26.134780,26.367489,26.490211,26.410786,26.119545,26.104902,25.948574,26.462763,26.300869
5,5,26.018894,26.118480,25.916462,26.133545,25.676086,25.401218,25.829260,25.753624,25.677235,...,25.794971,25.720280,25.998123,26.065067,26.020042,25.754292,25.673878,25.551908,26.050010,25.823908
6,6,26.184391,26.314167,26.096115,26.183678,25.776760,25.487400,26.005419,25.922762,25.892223,...,25.816332,25.888233,26.175346,26.117940,26.123539,25.870728,25.882206,25.717922,26.021633,26.001210
7,7,26.297771,26.457302,26.190557,26.221070,25.974550,25.505058,26.206623,26.093607,26.139915,...,26.153717,26.043394,26.331623,26.302353,26.334793,25.874570,25.984970,25.749758,26.218204,26.103132
8,8,26.326670,26.619793,26.256437,26.323599,25.983473,25.498980,26.337350,26.153553,26.184824,...,26.119747,25.923903,26.403278,26.470139,26.344688,26.014856,26.230545,25.956917,26.221523,26.151010
9,9,26.932737,27.187920,26.793346,26.783827,26.698051,25.905163,26.829552,26.651350,26.618565,...,26.577251,26.463078,26.826931,26.999010,26.745953,26.512848,26.575340,26.594208,26.591324,26.536510


In [115]:
logit_diffs = logit_diffs.T
logit_diffs

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
Unnamed: 0,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000
0,3.411159,-1.441616,-3.118092,26.490910,26.443226,26.018894,26.184391,26.297771,26.326670,26.932737,...,29.670404,30.202938,30.831140,30.383461,29.489765,28.851410,27.446210,21.535427,15.505719,-0.353825
1,3.411159,-1.566062,-3.236216,26.448448,26.418404,26.118480,26.314167,26.457302,26.619793,27.187920,...,29.020760,29.566803,30.165115,29.499105,29.334131,28.920383,27.233984,21.870222,15.498855,-0.351089
2,3.411159,-1.602333,-3.286282,26.365700,26.263874,25.916462,26.096115,26.190557,26.256437,26.793346,...,29.561316,29.585140,30.316254,29.856068,29.121174,28.193464,26.781834,21.111850,14.909542,-0.374791
3,3.411159,-1.448177,-3.232544,26.523554,26.482958,26.133545,26.183678,26.221070,26.323599,26.783827,...,28.400766,28.616251,29.369352,28.724320,27.895610,26.688896,25.488310,20.030650,13.736554,-0.421673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,3.411159,-1.539434,-3.256016,26.300419,26.119545,25.754292,25.870728,25.874570,26.014856,26.512848,...,28.018310,28.272871,28.949575,28.279550,27.452127,26.673458,25.256754,19.750347,13.943590,-0.412844
246,3.411159,-1.558260,-3.269503,26.154408,26.104902,25.673878,25.882206,25.984970,26.230545,26.575340,...,28.255507,28.757479,29.642424,28.870937,28.239498,27.538023,26.335192,21.059362,15.059591,-0.387747
247,3.411159,-1.658414,-3.175600,26.054653,25.948574,25.551908,25.717922,25.749758,25.956917,26.594208,...,28.458773,28.602953,29.359644,28.887665,28.285180,27.125526,25.722498,20.124197,14.283387,-0.387879
248,3.411159,-1.521465,-3.215193,26.567352,26.462763,26.050010,26.021633,26.218204,26.221523,26.591324,...,28.859468,29.449910,30.059040,29.886200,29.292250,28.320038,27.453220,21.994223,15.930881,-0.346668


In [33]:
unnamed_columns = [col for col in logit_diffs.columns if col.startswith('Unnamed')]

# Drop the unnamed columns from the DataFrame
df_cleaned = logit_diffs.drop(columns=unnamed_columns)
df_cleaned

# I need to 

,2,3,4,5,6,7,8,9,10,11,...,20,21,22,23,24,25,26,27,28,29
0,-0.855447,-1.755767,30.117956,30.082644,29.273748,28.785915,28.256042,27.723215,27.252330,26.851614,...,23.314410,23.129503,23.155931,23.082134,22.752913,22.606373,21.094387,16.652733,8.754633,-0.186035
1,-0.869199,-1.765149,30.205360,30.177765,29.346460,28.872427,28.303541,27.785967,27.382880,26.945087,...,22.788445,22.672010,22.738379,22.816319,22.474653,22.442192,20.907160,16.544323,8.795547,-0.187233
2,-0.885069,-1.785538,30.083944,30.029594,29.255096,28.890398,28.384726,27.891384,27.479591,27.055141,...,23.054434,23.043617,23.235836,23.090750,22.403824,22.075459,20.607582,15.882179,8.305014,-0.208775
3,-0.848781,-1.790689,30.316391,30.239521,29.470325,28.994738,28.371683,27.841646,27.329868,26.927244,...,23.042969,22.837975,23.000240,22.853916,22.424797,22.061684,20.799612,16.376795,8.580783,-0.204940
4,-0.888616,-1.801031,29.602118,29.463419,28.682140,28.208447,27.656708,27.174038,26.679783,26.353920,...,22.734324,22.616490,22.765144,22.669746,22.290602,22.149450,20.788479,16.011335,8.201328,-0.206898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-0.852389,-1.782505,29.959852,29.835320,29.100975,28.674656,28.124468,27.686684,27.170994,26.700602,...,22.785719,22.510450,22.620525,22.559008,21.955553,21.473595,20.257257,15.537518,7.633213,-0.239179
246,-0.879039,-1.816436,29.867983,29.762825,28.965391,28.579054,28.040289,27.655920,27.190208,26.814562,...,22.665812,22.468070,22.779400,22.444610,21.808187,21.494810,20.141464,15.831924,8.001150,-0.217457
247,-0.930633,-1.779508,29.634540,29.564570,28.791399,28.396688,27.808830,27.362675,26.861320,26.440040,...,22.529509,22.558992,22.388449,22.436861,21.992172,21.625895,20.156567,15.685631,7.960706,-0.217612
248,-0.887470,-1.764701,30.241081,30.104765,29.331427,28.861538,28.392620,27.881262,27.395102,26.959402,...,22.987173,22.987389,23.199968,23.198528,22.896930,22.653753,21.387054,16.815556,9.161243,-0.178931


In [116]:
# only for the 10 thing
# df_cleaned = df_cleaned.drop(df_cleaned.columns[[0, 1]], axis=1)
# df_cleaned = df_cleaned.drop(0)
# df_cleaned = df_cleaned.drop(1)

# df_cleaned = logit_diffs.drop(0)
# df_cleaned

df_cleaned = logit_diffs.drop(index='Unnamed: 0')
df_cleaned

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,3.411159,-1.441616,-3.118092,26.490910,26.443226,26.018894,26.184391,26.297771,26.326670,26.932737,...,29.670404,30.202938,30.831140,30.383461,29.489765,28.851410,27.446210,21.535427,15.505719,-0.353825
1,3.411159,-1.566062,-3.236216,26.448448,26.418404,26.118480,26.314167,26.457302,26.619793,27.187920,...,29.020760,29.566803,30.165115,29.499105,29.334131,28.920383,27.233984,21.870222,15.498855,-0.351089
2,3.411159,-1.602333,-3.286282,26.365700,26.263874,25.916462,26.096115,26.190557,26.256437,26.793346,...,29.561316,29.585140,30.316254,29.856068,29.121174,28.193464,26.781834,21.111850,14.909542,-0.374791
3,3.411159,-1.448177,-3.232544,26.523554,26.482958,26.133545,26.183678,26.221070,26.323599,26.783827,...,28.400766,28.616251,29.369352,28.724320,27.895610,26.688896,25.488310,20.030650,13.736554,-0.421673
4,3.411159,-1.618578,-3.253272,26.057621,25.997768,25.676086,25.776760,25.974550,25.983473,26.698051,...,29.151955,29.502697,30.358704,30.029476,29.333680,28.571476,27.471155,21.820194,15.925683,-0.338873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,3.411159,-1.539434,-3.256016,26.300419,26.119545,25.754292,25.870728,25.874570,26.014856,26.512848,...,28.018310,28.272871,28.949575,28.279550,27.452127,26.673458,25.256754,19.750347,13.943590,-0.412844
246,3.411159,-1.558260,-3.269503,26.154408,26.104902,25.673878,25.882206,25.984970,26.230545,26.575340,...,28.255507,28.757479,29.642424,28.870937,28.239498,27.538023,26.335192,21.059362,15.059591,-0.387747
247,3.411159,-1.658414,-3.175600,26.054653,25.948574,25.551908,25.717922,25.749758,25.956917,26.594208,...,28.458773,28.602953,29.359644,28.887665,28.285180,27.125526,25.722498,20.124197,14.283387,-0.387879
248,3.411159,-1.521465,-3.215193,26.567352,26.462763,26.050010,26.021633,26.218204,26.221523,26.591324,...,28.859468,29.449910,30.059040,29.886200,29.292250,28.320038,27.453220,21.994223,15.930881,-0.346668


In [117]:
df_cleaned.to_csv("figures/posvals_10context_norm", index=False)

# Analyze the accuracy metrics from the logit_diffs files

In [81]:
logit_diffs = pd.read_csv("figures/logit_diffs_01context")
logit_diffs

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.038448,-0.930255,-1.757942,25.878557,25.924210,25.114944,24.824290,24.542402,24.190693,23.760015,...,19.350807,19.357150,19.155642,19.280537,18.752617,18.467804,16.905539,12.443644,5.121335,-0.254215
1,0.038448,-0.933888,-1.706876,26.083668,26.104929,25.321503,24.913860,24.495197,24.145014,23.764893,...,20.036816,20.072930,20.331960,20.112993,19.753443,19.547740,17.844204,13.712940,6.192161,-0.217024
2,0.038448,-0.923910,-1.713627,23.830206,24.017073,23.429983,23.143389,22.884724,22.746817,22.508207,...,18.789145,18.827074,19.265629,19.341883,18.705640,18.489530,16.928074,12.583666,5.613316,-0.212200
3,0.038448,-0.873033,-1.717767,27.493036,27.497078,26.619562,26.157734,25.649216,25.272448,24.901323,...,20.945498,20.888727,21.120155,20.846310,19.946009,19.502980,17.836400,14.243299,6.652761,-0.230651
4,0.038448,-0.926325,-1.744440,25.209446,25.123234,24.482937,23.996162,23.690859,23.375471,22.847837,...,17.759813,18.059761,18.287918,18.457426,18.255697,17.660254,16.669731,12.264782,4.851778,-0.267455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.038448,-0.916115,-1.745319,26.036823,25.886274,25.195753,24.878603,24.519876,24.105085,23.745306,...,20.023706,19.712273,19.677422,19.412600,18.650648,18.381838,17.237930,13.021439,5.429454,-0.257781
246,0.038448,-0.925542,-1.735827,24.746292,24.858315,24.101536,23.825428,23.590775,23.249088,23.019207,...,19.711937,19.779943,19.360144,18.872196,17.898272,17.758436,16.405080,12.031109,5.028726,-0.262228
247,0.038448,-0.948089,-1.715012,26.251657,26.232946,25.459055,25.070879,24.703220,24.454268,24.126942,...,20.960136,20.986599,20.856285,20.989790,20.287258,19.866170,18.401348,13.984354,6.757117,-0.208326
248,0.038448,-0.920943,-1.699773,25.702902,25.655752,24.819595,24.437370,24.103800,23.786686,23.376083,...,18.990469,19.235764,19.026411,19.019209,18.670155,18.513840,16.875240,12.843174,5.705345,-0.238517


In [82]:
logit_diffs.mean(axis=0).values

array([ 0.03844833, -0.91186349, -1.74352802, 26.12242418, 26.099282  ,
       25.36685334, 24.96684822, 24.60525811, 24.260306  , 23.81748958,
       23.51497714, 23.0407081 , 22.59874177, 22.0558913 , 21.66974001,
       21.12284249, 20.73892395, 20.41284645, 20.21492526, 19.77861417,
       19.71818935, 19.7942917 , 19.60133488, 19.05423646, 18.87183619,
       17.51521439, 13.18532114,  5.87847991, -0.23968347])

In [83]:
# find accuracy normalized to the median logit difference
# for each layer, find the median logit difference
medians = logit_diffs.median(axis=0).values
medians

array([ 0.03844833, -0.91328177, -1.7449711 , 26.1979295 , 26.160128  ,
       25.412638  , 25.0153095 , 24.6626985 , 24.3303405 , 23.880534  ,
       23.587326  , 23.127869  , 22.647568  , 22.1285215 , 21.751631  ,
       21.170766  , 20.8110055 , 20.4116205 , 20.2150535 , 19.8445805 ,
       19.760273  , 19.830141  , 19.6060615 , 19.0406885 , 18.8733795 ,
       17.513816  , 13.148142  ,  5.8951903 , -0.23849765])

In [84]:
import torch
def binary_accuracy(tensor1, tensor2):
    assert tensor1.shape == tensor2.shape, "Input tensors must have the same shape"
    correct_predictions = torch.eq(tensor1, tensor2).sum().item()
    total_elements = tensor1.numel()
    accuracy = correct_predictions / total_elements
    return accuracy

In [85]:
num_layers = 28
norm_accs = []
for i in range(num_layers):
    predictions = np.where((logit_diffs[logit_diffs.columns[i]].values) > (medians[i]), 1, 0)
    # predictions = np.where((predictions == 1) & (logit_diffs[logit_diffs.columns[i]].values < 0), 0, 1)
    acc = binary_accuracy(torch.Tensor(predictions), torch.Tensor(y[250:]))
    norm_accs.append(acc)
print(norm_accs)

[0.532, 0.472, 0.416, 0.536, 0.536, 0.536, 0.536, 0.52, 0.528, 0.512, 0.496, 0.504, 0.512, 0.496, 0.504, 0.536, 0.496, 0.48, 0.504, 0.496, 0.504, 0.52, 0.488, 0.504, 0.512, 0.496, 0.528, 0.544]


# Analyze the accuracy from the pos_vals files

In [118]:
posvals = pd.read_csv("figures/posvals_10context_norm")
posvals

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,3.411159,-1.441616,-3.118092,26.490910,26.443226,26.018894,26.184391,26.297771,26.326670,26.932737,...,29.670404,30.202938,30.831140,30.383461,29.489765,28.851410,27.446210,21.535427,15.505719,-0.353825
1,3.411159,-1.566062,-3.236216,26.448448,26.418404,26.118480,26.314167,26.457302,26.619793,27.187920,...,29.020760,29.566803,30.165115,29.499105,29.334131,28.920383,27.233984,21.870222,15.498855,-0.351089
2,3.411159,-1.602333,-3.286282,26.365700,26.263874,25.916462,26.096115,26.190557,26.256437,26.793346,...,29.561316,29.585140,30.316254,29.856068,29.121174,28.193464,26.781834,21.111850,14.909542,-0.374791
3,3.411159,-1.448177,-3.232544,26.523554,26.482958,26.133545,26.183678,26.221070,26.323599,26.783827,...,28.400766,28.616251,29.369352,28.724320,27.895610,26.688896,25.488310,20.030650,13.736554,-0.421673
4,3.411159,-1.618578,-3.253272,26.057621,25.997768,25.676086,25.776760,25.974550,25.983473,26.698051,...,29.151955,29.502697,30.358704,30.029476,29.333680,28.571476,27.471155,21.820194,15.925683,-0.338873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,3.411159,-1.539434,-3.256016,26.300419,26.119545,25.754292,25.870728,25.874570,26.014856,26.512848,...,28.018310,28.272871,28.949575,28.279550,27.452127,26.673458,25.256754,19.750347,13.943590,-0.412844
246,3.411159,-1.558260,-3.269503,26.154408,26.104902,25.673878,25.882206,25.984970,26.230545,26.575340,...,28.255507,28.757479,29.642424,28.870937,28.239498,27.538023,26.335192,21.059362,15.059591,-0.387747
247,3.411159,-1.658414,-3.175600,26.054653,25.948574,25.551908,25.717922,25.749758,25.956917,26.594208,...,28.458773,28.602953,29.359644,28.887665,28.285180,27.125526,25.722498,20.124197,14.283387,-0.387879
248,3.411159,-1.521465,-3.215193,26.567352,26.462763,26.050010,26.021633,26.218204,26.221523,26.591324,...,28.859468,29.449910,30.059040,29.886200,29.292250,28.320038,27.453220,21.994223,15.930881,-0.346668


In [119]:
medians = posvals.median(axis=0).values
medians

array([ 3.4111586 , -1.5229759 , -3.218979  , 26.3586885 , 26.2586715 ,
       25.8542315 , 26.0015165 , 26.1509385 , 26.2413045 , 26.7141285 ,
       26.978942  , 27.845909  , 27.807969  , 28.123562  , 27.8595275 ,
       28.2588865 , 29.0438255 , 29.332281  , 28.102395  , 29.0034925 ,
       29.137896  , 29.892045  , 29.4854505 , 28.9657865 , 28.13838   ,
       26.8416725 , 21.253811  , 15.1593785 , -0.36214967])

In [120]:
import torch
def binary_accuracy(tensor1, tensor2):
    assert tensor1.shape == tensor2.shape, "Input tensors must have the same shape"
    correct_predictions = torch.eq(tensor1, tensor2).sum().item()
    total_elements = tensor1.numel()
    accuracy = correct_predictions / total_elements
    return accuracy

In [121]:
num_layers = 29
norm_accs = []
for i in range(num_layers):
    predictions = np.where((posvals[posvals.columns[i]].values) > (medians[i]), 1, 0)
    # predictions = np.where((predictions == 1) & (logit_diffs[logit_diffs.columns[i]].values < 0), 0, 1)
    acc = binary_accuracy(torch.Tensor(predictions), torch.Tensor(y[250:]))
    norm_accs.append(acc)
print(norm_accs)

[0.532, 0.56, 0.488, 0.544, 0.552, 0.544, 0.552, 0.568, 0.6, 0.584, 0.52, 0.528, 0.52, 0.536, 0.552, 0.528, 0.536, 0.512, 0.512, 0.52, 0.496, 0.496, 0.488, 0.488, 0.52, 0.512, 0.528, 0.536, 0.568]
